# Step-by-step — Cracking a test password locally 

> **Important:** everything below is done **locally on your Kali VM** against a test file you created. Only use these techniques on systems you own or have explicit written permission to test.

---

## 1) Ensure John is installed

```bash
which john || sudo apt install -y john
```

---

## 2) Generate a sha512crypt hash of the test password

Use single quotes (or escape `!`) to avoid zsh history expansion:

```bash
HASH=$(mkpasswd -m sha-512 'PasswordTest123!')
```

---

## 3) Create the `user:hash` input file for John

```bash
echo "miky:$HASH" > ~/testshadow.txt
cat ~/testshadow.txt
# expected: miky:$6$... (sha512crypt format)
```

---

## 4) Create a small wordlist that includes the test password

```bash
printf 'PasswordTest123!\npassword\n123456\n' > ~/mylist.txt
cat ~/mylist.txt
```

---

## 5) Run John the Ripper (wordlist attack)

```bash
john --wordlist=~/mylist.txt --format=sha512crypt ~/testshadow.txt
```

---

## 6) Show cracked passwords

```bash
john --show ~/testshadow.txt
```

You should see the username and the recovered password (because it was in the wordlist).

---

## 7) Monitor status (optional)

```bash
john --status
```

---

## 8) Cleanup (remove test files and John’s pot file)

```bash
rm -f ~/testshadow.txt ~/mylist.txt
rm -f ~/.john/john.pot
# optional secure delete
shred -u ~/testshadow.txt ~/mylist.txt ~/.john/john.pot 2>/dev/null || true
```

---

## Quick notes / troubleshooting

* In **zsh** the `!` character triggers history expansion. Use single quotes (`'PasswordTest123!'`) or escape (`PasswordTest123\!`) when passing passwords to avoid `zsh: event not found` or continuation prompts like `dquote cmdsubst>`.
* The hash format `sha512crypt` matches Linux `/etc/shadow` entries (prefixed with `$6$`).
* This exercise is **offline/local** and safe for learning. Do **not** use these tools against systems you do not own or lack written authorization to test.
